In [ ]:
# https://chatgpt.com/share/677ea92d-b844-8009-91d1-932799305a61

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Funzione per calcolare i minimi e massimi locali
def find_local_extrema(series):
    """Trova i minimi e i massimi locali in una serie temporale."""
    minima = (series < series.shift(1)) & (series < series.shift(-1))
    maxima = (series > series.shift(1)) & (series > series.shift(-1))
    return minima, maxima

# Funzioni che mappano la variazione percentuale alla percentuale da vendere
def map_sell_percentage(price_change_percentage):
    """Esempio di funzione di mappatura."""
    return max(0, min(1, price_change_percentage))  # Limita tra 0% e 100%

def backtest_strategy(crypto_prices, initial_cash, initial_allocations, timeframe, window, 
                      perc_sell_ETH_BTC, perc_sell_BTC_ETH, include_liquidity):
    """Backtest della strategia di rotazione del capitale."""

    # Inizializzazione variabili
    cash = initial_cash
    holdings = {
        'BTC': cash * initial_allocations['BTC'] / crypto_prices['BTC'].iloc[0],
        'ETH': cash * initial_allocations['ETH'] / crypto_prices['ETH'].iloc[0],
        'cash': cash * (1 - sum(initial_allocations.values()))
    }

    # Calcolo del rapporto BTC/ETH
    crypto_prices['BTC_ETH_ratio'] = crypto_prices['BTC'] / crypto_prices['ETH']

    # Media mobile del rapporto
    crypto_prices['ratio_moving_avg'] = crypto_prices['BTC_ETH_ratio'].rolling(window=window).mean()

    # Identifica minimi e massimi locali della media mobile
    minima, maxima = find_local_extrema(crypto_prices['ratio_moving_avg'])

    last_extreme = None  # Tiene traccia dell'ultimo estremo ("minimo" o "massimo")
    last_extreme_value = None

    # Iterazione sui dati con intervallo definito da timeframe
    for i in range(0, len(crypto_prices), timeframe):
        if i + window - 1 >= len(crypto_prices):
            break  # Esce se la finestra supera il range dei dati

        # current_ratio = crypto_prices['BTC_ETH_ratio'].iloc[i]
        moving_avg = crypto_prices['ratio_moving_avg'].iloc[i]

        # Determina il tipo di estremo più recente
        if minima.iloc[i]:
            last_extreme = 'min'
            last_extreme_value = moving_avg
        elif maxima.iloc[i]:
            last_extreme = 'max'
            last_extreme_value = moving_avg

        # Strategia di rotazione
        if last_extreme == 'max':
            price_drop_percentage = (last_extreme_value - moving_avg) / last_extreme_value
            sell_percentage = map_sell_percentage(price_drop_percentage)

            eth_to_sell = sell_percentage * holdings['ETH']
            sell_value = eth_to_sell * crypto_prices['ETH'].iloc[i]

            holdings['ETH'] -= eth_to_sell
            if include_liquidity:
                buy_value = sell_value * perc_sell_ETH_BTC / (sell_value + holdings['cash'])
            else:
                buy_value = sell_value * perc_sell_ETH_BTC

            holdings['BTC'] += buy_value / crypto_prices['BTC'].iloc[i]
            holdings['cash'] += sell_value * (1 - perc_sell_ETH_BTC)

        elif last_extreme == 'min':
            price_rise_percentage = (moving_avg - last_extreme_value) / last_extreme_value
            sell_percentage = map_sell_percentage(price_rise_percentage)

            btc_to_sell = sell_percentage * holdings['BTC']
            sell_value = btc_to_sell * crypto_prices['BTC'].iloc[i]

            holdings['BTC'] -= btc_to_sell
            if include_liquidity:
                buy_value = sell_value * perc_sell_BTC_ETH / (sell_value + holdings['cash'])
            else:
                buy_value = sell_value * perc_sell_BTC_ETH

            holdings['ETH'] += buy_value / crypto_prices['ETH'].iloc[i]
            holdings['cash'] += sell_value * (1 - perc_sell_BTC_ETH)

    # Calcolo del valore finale del portafoglio
    final_value = (holdings['BTC'] * crypto_prices['BTC'].iloc[-1] +
                   holdings['ETH'] * crypto_prices['ETH'].iloc[-1] +
                   holdings['cash'])

    return final_value, holdings

In [ ]:
# Esempio di utilizzo
crypto_prices = pd.DataFrame({
    'BTC': np.random.uniform(20000, 25000, 100),
    'ETH': np.random.uniform(1000, 2000, 100)
})

initial_cash = 10000
initial_allocations = {'BTC': 0.5, 'ETH': 0.4}
timeframe = 7
window = 14
perc_sell_ETH_BTC = 0.5
perc_sell_BTC_ETH = 0.5
include_liquidity = True

final_value, final_holdings = backtest_strategy(
    crypto_prices, initial_cash, initial_allocations, timeframe, window,
    perc_sell_ETH_BTC, perc_sell_BTC_ETH, include_liquidity
)

print("Valore finale del portafoglio:", final_value)
print("Allocazioni finali:", final_holdings)